In [18]:
puzzle = open("AoC24.txt").read().splitlines()

In [19]:
xBits = {}
yBits = {}
zBits = {}
gates = {}
gateRules = {}
i = 0
while puzzle[i]!="":
    gate, value = puzzle[i].split(": ")
    if gate[0]=="x":
        xBits[gate] = int(value)
    elif gate[0]=="y":
        yBits[gate] = int(value)
    gates[gate] = value
    i+=1
i+=1
while i<len(puzzle):
    inGates, outGate = puzzle[i].split(" -> ")
    inGate1, op, inGate2 = inGates.split(" ")
    gateRules[outGate] = (inGate1, op, inGate2)
    i+=1

In [20]:
z_outs = []
for out in gateRules:
    if out[0]=="z":
        z_outs.append(out)
z_outs.sort()
def getVal(out, gateRule, gates, cache):
    if out in cache:
        gates[out] = cache[out]
        return cache[out], cache
    if out in gates:
        cache[out] = int(gates[out])
        return cache[out], cache
    inGate1, op, inGate2 = gateRule[out]
    if op=="AND":
        cache[out] = getVal(inGate1, gateRule, gates, cache)[0] & getVal(inGate2, gateRule, gates, cache)[0]
    elif op=="OR":
        cache[out] = getVal(inGate1, gateRule, gates, cache)[0] | getVal(inGate2, gateRule, gates, cache)[0]
    elif op=="XOR":
        cache[out] = getVal(inGate1, gateRule, gates, cache)[0] ^ getVal(inGate2, gateRule, gates, cache)[0]
    gates[out] = cache[out]
    return cache[out], cache

In [21]:
ans1 = 0
for out in z_outs:
    num = int(out[1:])
    ans1 = getVal(out, gateRules, gates, {})[0]*2**num + ans1
print(ans1)


38869984335432


In [22]:
invalidEnds = []
#checks if there are any rules that are x__ OP y__ -> z__ where OP isn't XOR (indicates wrong rule)
for out in gateRules:
    if out[0]=="z":
        in1, op, in2 = gateRules[out]
        if op!="XOR":
            num = int(out[1:])
            if num!=45:
                invalidEnds.append((out, gateRules[out]))
            if num==45 and op!="OR":
                invalidEnds.append((out, gateRules[out]))

In [23]:
invalidMids = []
#checks if there are any rules that have XOR but aren't start or end
for out in gateRules:
    if out[0]!="z":
        in1, op, in2 = gateRules[out]
        if op=="XOR":
            if in1[0] not in "xy" or in2[0] not in "xy":
                invalidMids.append((out, gateRules[out]))

In [24]:
def getCorrectEndFromMid(invalidMid, gateRules):
    left, op, right = gateRules[invalidMid]
    left1, opL, right1 = gateRules[left]
    if opL=="XOR":
        return left1[1:]
    left2, opR, right2 = gateRules[right]
    if opR=="XOR":
        return left2[1:]
    return -1

In [25]:
newGateRules = {}
for out in gateRules:
    if out not in invalidEnds and out not in invalidMids:
        newGateRules[out] = gateRules[out]
for out in invalidMids:
    mid = out[0]
    act = "z"+getCorrectEndFromMid(mid, gateRules)
    newGateRules[act] = gateRules[mid]
    newGateRules[mid] = gateRules[act]

In [26]:
i = 0
newGates = {}
while puzzle[i]!="":
    gate, value = puzzle[i].split(": ")
    newGates[gate] = value
    i+=1

In [27]:
x_val = 0
y_val = 0
for x, val in xBits.items():
    x_val += val*2**int(x[1:])
for y, val in yBits.items():
    y_val += val*2**int(y[1:])
new_z = 0
for out in z_outs:
    num = int(out[1:])
    val1 = getVal(out, newGateRules, newGates, {})[0]
    new_z += val1*2**num
binRep = bin((x_val+y_val)^new_z)[2:][::-1]
firstMessedBit = 0
for i in range(46):
    if binRep[i]=="1":
        firstMessedBit = i
        break
strMessed = str(firstMessedBit)
if len(strMessed)==1:
    strMessed = "0"+strMessed
invalidCarries = []
for out in gateRules:
    in1, op, in2 = gateRules[out]
    if in1 == "x"+strMessed or in1 == "y"+strMessed:
        invalidCarries.append((out, gateRules[out]))

In [28]:
invalid = []
for out,rule in invalidEnds:
    invalid.append(out)
for out, rule in invalidMids:
    invalid.append(out)
for out, rule in invalidCarries:
    invalid.append(out)
invalid.sort()
output = ""
for c in invalid:
    output += c+","
ans2 = output[:-1]
print(ans2)

drg,gvw,jbp,jgc,qjb,z15,z22,z35
